In [ ]:
!pip install transformers
!pip install datasets
!pip install peft
!pip install accelerate
from transformers import Trainer, TrainingArguments
from transformers import AutoModelForCausalLM, AutoTokenizer, default_data_collator, get_linear_schedule_with_warmup
from peft import get_peft_config, get_peft_model, PromptTuningInit, PromptTuningConfig, TaskType, PeftType
import torch
from datasets import load_dataset
import os
from torch.utils.data import DataLoader
from tqdm import tqdm


In [ ]:
# !pip install accelerate -U

In [ ]:
device = "mps"
# device = "cuda"
model_name_or_path = "andreaskoepf/pythia-1.4b-gpt4all-pretrain"
tokenizer_name_or_path = "andreaskoepf/pythia-1.4b-gpt4all-pretrain"
peft_config = PromptTuningConfig(
    task_type=TaskType.CAUSAL_LM,
    prompt_tuning_init=PromptTuningInit.TEXT,
    num_virtual_tokens=8,
    prompt_tuning_init_text="Classify if the tweet is a complaint or not:",
    tokenizer_name_or_path=tokenizer_name_or_path,
)

dataset_name = "twitter_complaints"
text_column = "Tweet text"
label_column = "text_label"
max_length = 100
learning_rate = 3e-2
num_epochs = 100
batch_size = 8
output_dir = './output'

# 1. load a subset of the RAFT dataset at https://huggingface.co/datasets/ought/raft
dataset = load_dataset("ought/raft", dataset_name)

# get lable's possible values
label_values = [name.replace("_", "") for name in dataset["train"].features["Label"].names]
# append label value to the dataset to make it more readable
dataset = dataset.map(
    lambda x: {label_column: [label_values[label] for label in x["Label"]]},
    batched=True,
    num_proc=1
)
# have a look at the data structure
dataset["train"][0]

  0%|          | 0/2 [00:00<?, ?it/s]

{'Tweet text': '@HMRCcustomers No this is my first job',
 'ID': 0,
 'Label': 2,
 'text_label': 'no complaint'}

In [ ]:
# 2. dataset
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path)
if tokenizer.pad_token_id is None:
    tokenizer.pad_token_id = tokenizer.eos_token_id

def preprocess_fn(examples):
    tweets = examples[text_column]
    # pad labels with a pad token at the end
    labels = [str(x) + tokenizer.pad_token for x in examples[label_column]]
    # concatenate the tweet with it label
    inputs = [f"{text_column} : {tweet}\nLabel :{label}"
              for tweet, label in zip(tweets, labels)]
    # tokenize input
    model_inputs = tokenizer(inputs,
                           padding='max_length',
                           max_length=max_length,
                           truncation=True,)
    # tokenize label, as -100 not a valid token id, do the padding manually here
    labels_input_ids = []
    for i in range(len(labels)):
        ids = tokenizer(labels[i])["input_ids"]
        padding = [-100] * (max_length - len(ids))
        labels_input_ids.append(padding + ids)
        model_inputs["labels"] = labels_input_ids
        # make model inputs tensor
        model_inputs["input_ids"] = [torch.tensor(ids) for ids in model_inputs["input_ids"]]
        model_inputs["attention_mask"] = [torch.tensor(ids) for ids in model_inputs["attention_mask"]]
        model_inputs["labels"] = [torch.tensor(ids) for ids in model_inputs["labels"]]

    return model_inputs

# have a look at the preprocessing result
# print(preprocess_fn(dataset["train"][:2]))

processed_datasets = dataset.map(
    preprocess_fn,
    batched=True,
    num_proc=1,
    remove_columns=dataset["train"].column_names, #remove unprocessed column for training
    load_from_cache_file=False,
    desc="Running tokenizer on datasset"
)

test_size = round(len(processed_datasets["train"]) * 0.2)
train_val = processed_datasets["train"].train_test_split(
    test_size=test_size, shuffle=True, seed=42)
train_data = train_val["train"]
val_data = train_val["test"]


# 3. model
model = AutoModelForCausalLM.from_pretrained(model_name_or_path)
model = get_peft_model(model, peft_config)
print(model.print_trainable_parameters())
# trainable params: 8192 || all params: 559222784 || trainable%: 0.0014648902430985358

Running tokenizer on datasset:   0%|          | 0/50 [00:00<?, ? examples/s]

<ipython-input-16-e1f4fa4082c7>:26: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  model_inputs["input_ids"] = [torch.tensor(ids) for ids in model_inputs["input_ids"]]
<ipython-input-16-e1f4fa4082c7>:27: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  model_inputs["attention_mask"] = [torch.tensor(ids) for ids in model_inputs["attention_mask"]]


Running tokenizer on datasset:   0%|          | 0/3399 [00:00<?, ? examples/s]

trainable params: 16384 || all params: 1414598656 || trainable%: 0.0011582083674763508
None


In [ ]:
# 4. trainer
from transformers import Trainer, TrainingArguments
trainer = Trainer(
    model=model,
    train_dataset=train_data,
    eval_dataset=val_data,
    data_collator=default_data_collator,
    args=TrainingArguments(
      output_dir='./output',
      per_device_train_batch_size=batch_size,
      num_train_epochs=num_epochs,
      learning_rate=learning_rate,
      load_best_model_at_end=True,
      logging_strategy='steps',
      logging_steps=10,
      evaluation_strategy='steps',
      eval_steps=10,
      save_strategy='steps',
      save_steps=10,
    )
  )
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss,Validation Loss
10,7.278000,4.527699
20,3.849600,2.639633
30,2.640300,2.077576
40,1.728700,1.474593
50,1.217800,1.061706
60,0.884200,0.776880
70,0.644300,0.579637
80,0.483000,0.447307
90,0.381300,0.363490
100,0.311800,0.306514


TrainOutput(global_step=500, training_loss=0.4935280463695526, metrics={'train_runtime': 707.7569, 'train_samples_per_second': 5.652, 'train_steps_per_second': 0.706, 'total_flos': 3147821875200000.0, 'train_loss': 0.4935280463695526, 'epoch': 100.0})

In [ ]:
trainer.save_model("/content/model/prompt_tuning")

In [ ]:
# prompt_tuned_model = None

In [ ]:
# # 5. inference
# device = "cuda"
# def  inference():
#     def generate(inputs, infer_model):
#         with torch.no_grad():
#             inputs = {k: v.to(device) for k, v in inputs.items()}
#             outputs = infer_model.generate(
#                 input_ids=inputs["input_ids"],
#                 attention_mask=inputs["attention_mask"],
#                 max_new_tokens=20,
#                 eos_token_id=3
#             )
#             print(tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True)[0])

#     # # (1) base model_inference
#     # base_model = AutoModelForCausalLM.from_pretrained(model_name_or_path)
#     # base_model.to(device)
#     # inputs = tokenizer(
#     #     f'{text_column} : {"@denny the grocery price is soaring, even milk is becoming unaffordable, could you do something?"}\nLabel :',
#     #     return_tensors="pt",  # Return PyTorch torch.Tensor objects.
#     # )
#     # generate(inputs, base_model)
#     # print("----------------------------------------")
#     # shot1 = f'{text_column} : {"@nationalgridus I have no water and the bill is current and paid. Can you do something about this?"}\nLabel :complaint\n'
#     # shot2 = f'{text_column} : {"@HMRCcustomers No this is my first job"}\nLabel :no complaint\n'
#     # input = f'{text_column} : {"@denny the grocery price is soaring, even milk is becoming unaffordable, could you do something?"}\nLabel :'
#     # inputs_few_shot = tokenizer(
#     #     shot1 + shot2 + input,
#     #     return_tensors="pt",
#     # )
#     # generate(inputs_few_shot, base_model)

#     # (2) prompt-tuned model_inference
#     from peft import PeftModel, PeftConfig
#     path = "/content/model/prompt_tuning"
#     config = PeftConfig.from_pretrained(path)
#     pretrained_model = AutoModelForCausalLM.from_pretrained(config.base_model_name_or_path)
#     prompt_tuned_model = PeftModel.from_pretrained(pretrained_model, path)
#     prompt_tuned_model.to(device)
#     inputs = tokenizer(
#         f'{text_column} : {"@denny the grocery price is soaring, even milk is becoming unaffordable, could you do something?"}\nLabel :',
#         return_tensors="pt",  # Return PyTorch torch.Tensor objects.
#     )
#     generate(inputs, prompt_tuned_model)

# inference()

In [ ]:
# 参考链接：https://blog.csdn.net/qq_35812205/article/details/131647749